# Data Preprocessing
1. Topic Extraction
2. User Buckets
3. Data Split

## 데이터 로드

In [2]:
import pandas as pd
COLS = ["text_tokens", "hashtags", "tweet_id", "present_media", 
                      "present_links", "present_domains", "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id",
                     "engaged_with_user_follower_count", "engaged_with_user_following_count", 
                     "engaged_with_user_is_verified", "engaged_with_user_account_creation", "engaging_user_id",
                     "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified",
                     "engaging_user_account_creation", "engagee_follows_engager", "reply_timestamp",
                     "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
df = pd.read_csv('train1000.csv', names=COLS, skipinitialspace=True, skiprows=1)

In [3]:
df

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,101\t10117\t140\t119\t142\t119\t152\t119\t1010...,NaN,373C0F43762B7CEC1D75728BE8A33891,NaN,A2CE3A1941BA410A1C31496C355EFCD7,E14AF8A8D257BB47587843FE7D08382B,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1582126349,2A8B6AD2B9D55F535C2441AB673133D2,...,00000865A1538142CDA5936B07FE4311,65,166,False,1452599043,False,NaN,NaN,NaN,NaN
1,101\t10105\t10817\t10124\t59232\t18121\t15629\...,NaN,773A92D9E4824D06105C02BD044BB20A,NaN,NaN,NaN,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581971193,950A95B81407F33C412E520BE55A1450,...,000009A057792FF118B9E3F2578B8407,1814,1314,False,1322868747,True,1.581979e+09,NaN,NaN,NaN
2,101\t48561\t10116\t67737\t18554\t36371\t10989\...,NaN,218A6C27871801759F7380D7C41694A6,NaN,5C683B5A29B308CADD0D7EFA7C9C32D3,6717B03E03DEE1D7ACAE37649ACA7BD6,TopLevel,9BF3403E0EB7EA8A256DA9019C0B0716,1582047119,ABB2F7F22C34057BC7B30D627B0C137A,...,00000DEF82BE9EB5CFD07FB7DB94317B,4,73,False,1573996260,False,NaN,NaN,NaN,NaN
3,101\t100055\t69940\t10414\t159\t11305\t11166\t...,NaN,AB817EBA68064A0C8CBF4A6C059D92DC,Photo,E925556EE312213AD98C4D9F131D7A8D,D722330FEBEAAE68B4F4339CE8BD7C70,TopLevel,691890251F2B9FF922BE6D3699ABEFD2,1581554925,03F96C3B7CE2179B6347AA395880C963,...,0000109A57AFA64758EE4AAE2A01BFC7,15,124,False,1385502405,True,NaN,NaN,NaN,NaN
4,101\t62154\t32221\t71843\t10143\t10237\t15507\...,NaN,349120C1E2801857530393F16D4653A5,NaN,NaN,NaN,TopLevel,9BF3403E0EB7EA8A256DA9019C0B0716,1581568955,E035DCB47CB3DF98C5CD7CFEEC3BC704,...,000012366528B5FEE179A9606DBC9826,1226,655,False,1268639592,True,1.581570e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,101\t13069\t10161\t117\t12153\t11293\t10911\t1...,NaN,E05C04D566AC054E3C9C5BA6CC594ADD,NaN,NaN,NaN,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581976704,D3A922E971AFA7F40E0E4495497E7A17,...,000CE8F896EB59B4AD6452EE42DF8C0B,29,374,False,1381114187,False,NaN,NaN,NaN,NaN
996,101\t11045\t18561\t10132\t32461\t10369\t10121\...,NaN,6A418A8150BA118D95B8181A2E5C95E2,Photo,NaN,NaN,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,1582052151,40474075B5F45D4E4571494E967BB0B8,...,000CED9874136BDB73EE2E5507E89A85,3,74,False,1570506912,False,NaN,NaN,NaN,1.582053e+09
997,101\t17116\t14499\t10106\t10151\t24443\t10142\...,NaN,2D273BB12F094B420832490F2E52905F,NaN,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581862080,32F7913B6955BABDA1AD25833BBEC9C1,...,000CF188DCD86CC025395E44676AFEDD,24,252,False,1465667497,False,NaN,NaN,NaN,NaN
998,101\t15198\t16903\t14120\t131\t120\t120\t188\t...,NaN,D5D4411CA11166600AC0C9B1498FBA4E,Photo\tPhoto,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581737382,A164FFF71F170132EA0CBC88176D912D,...,000CF188DCD86CC025395E44676AFEDD,24,252,False,1465667497,False,NaN,NaN,NaN,1.581739e+09
